In [27]:
import os
import pandas as pd

In [31]:
# Load file
pfas_data_AL_LA = pd.read_csv(r"data\pfas_data\UCMR5_All_Tribes_AK_LA.txt", delimiter="\t", encoding="cp1252")

# Filter out rows where PWSID is numeric (tribal data has numeric identifiers)
pfas_data_AL_LA = pfas_data_AL_LA[~pfas_data_AL_LA['PWSID'].str.isnumeric()]

# Extract state abbreviation (first two characters of PWSID)
pfas_data_AL_LA['State'] = pfas_data_AL_LA['PWSID'].str[:2]

# Keep only relevant columns: State and AnalyticalResultValue
pfas_data_cleaned_AL_LA = pfas_data_AL_LA[['State', 'AnalyticalResultValue', 'MRL']]

# Filter for relevant states (same list of states you provided before)
relevant_states = ['NY', 'NJ', 'CA', 'GA', 'FL', 'TX', 'MD', 'VA', 'NC', 'OH', 'IL', 'MI', 'PA', 'SC', 'CT', 'TN', 'AZ', 'MA', 'LA']
pfas_data_cleaned_AL_LA = pfas_data_cleaned_AL_LA[pfas_data_cleaned_AL_LA['State'].isin(relevant_states)]

# Mapping of state abbreviations to numeric state_id (based on your provided dictionary)
state_abbr_to_state_id = {
    'NY': 11, 'NJ': 10, 'CA': 1, 'GA': 4, 'FL': 3, 'TX': 17, 'MD': 7, 'VA': 18, 'NC': 12,
    'OH': 13, 'IL': 5, 'MI': 9, 'PA': 14, 'SC': 15, 'CT': 2, 'TN': 16, 'AZ': 0, 'MA': 8, 'LA': 6
}

# Apply the mapping to the 'State' column to create 'state_id'
pfas_data_cleaned_AL_LA['state_id'] = pfas_data_cleaned_AL_LA['State'].map(state_abbr_to_state_id)

# Handle below-MRL values (set to MRL)
pfas_data_cleaned_AL_LA['AnalyticalResultValue'] = pd.to_numeric(
    pfas_data_cleaned_AL_LA['AnalyticalResultValue'], errors='coerce'
).fillna(pfas_data_cleaned_AL_LA['MRL'])

# Calculate the average AnalyticalResultValue for each state
state_avg_arv_AL_LA = pfas_data_cleaned_AL_LA.groupby('state_id', as_index=False)['AnalyticalResultValue'].mean()

# Save the cleaned PFAS data with averages
state_avg_arv_AL_LA.to_csv(r"data/pfas_data/state_avg_arv_AL_LA.csv", index=False)
print("State average AnalyticalResultValues for AL/LA saved as 'state_avg_arv_AL_LA.csv'.")
print(pfas_data_cleaned_AL_LA.head())


State average AnalyticalResultValues for AL/LA saved as 'state_avg_arv_AL_LA.csv'.
      State  AnalyticalResultValue    MRL  state_id
45410    AZ                  0.005  0.005         0
45411    AZ                  0.004  0.004         0
45412    AZ                  0.002  0.002         0
45413    AZ                  0.020  0.020         0
45414    AZ                  0.003  0.003         0


In [28]:
pfas_data = pd.read_csv(r"data/pfas_data/UCMR5_All_MA_WY.txt", delimiter="\t", encoding="cp1252")

# Extract state abbreviation
pfas_data['State'] = pfas_data['PWSID'].str[:2]

# Keep only relevant columns: State and AnalyticalResultValue
pfas_data_cleaned = pfas_data[['State', 'AnalyticalResultValue', 'MRL']]

# Placeholder: Filter for relevant states
relevant_states = ['NY', 'NJ', 'CA', 'GA', 'FL', 'TX', 'MD', 'VA', 'NC', 'OH', 'IL', 'MI', 'PA', 'SC', 'CT', 'TN', 'AZ', 'MA', 'LA']
pfas_data_cleaned = pfas_data_cleaned[pfas_data_cleaned['State'].isin(relevant_states)]

# Mapping of state abbreviations to numeric state_id (based on your provided dictionary)
state_abbr_to_state_id = {
    'NY': 11, 'NJ': 10, 'CA': 1, 'GA': 4, 'FL': 3, 'TX': 17, 'MD': 7, 'VA': 18, 'NC': 12,
    'OH': 13, 'IL': 5, 'MI': 9, 'PA': 14, 'SC': 15, 'CT': 2, 'TN': 16, 'AZ': 0, 'MA': 8, 'LA': 6
}

# Apply the mapping to the 'State' column to create 'state_id'
pfas_data_cleaned['state_id'] = pfas_data_cleaned['State'].map(state_abbr_to_state_id)

# Handle below-MRL values (set to MRL)
pfas_data_cleaned['AnalyticalResultValue'] = pd.to_numeric(
    pfas_data_cleaned['AnalyticalResultValue'], errors='coerce'
).fillna(pfas_data_cleaned['MRL'])

# Calculate the average AnalyticalResultValue for each state
state_avg_arv = pfas_data_cleaned.groupby('state_id', as_index=False)['AnalyticalResultValue'].mean()

# Save the cleaned PFAS data with averages
state_avg_arv.to_csv(r"data/pfas_data/state_avg_arv.csv", index=False)
print("State average AnalyticalResultValues saved as 'state_avg_arv.csv'.")


State average AnalyticalResultValues saved as 'state_avg_arv.csv'.


In [29]:
print(pfas_data_cleaned.head)

<bound method NDFrame.head of        State  AnalyticalResultValue    MRL  state_id
0         MA                  0.003  0.003         8
1         MA                  0.003  0.003         8
2         MA                  0.003  0.003         8
3         MA                  0.003  0.003         8
4         MA                  0.003  0.003         8
...      ...                    ...    ...       ...
487356    VA                  0.005  0.005        18
487357    VA                  0.002  0.002        18
487358    VA                  0.004  0.004        18
487359    VA                  0.003  0.003        18
487360    VA                  0.005  0.005        18

[349705 rows x 4 columns]>


In [32]:
# Load the first PFAS dataset (state_avg_arv)
state_avg_arv = pd.read_csv(r"data/pfas_data/state_avg_arv.csv")

# Load the second PFAS dataset (state_avg_arv_AL_LA)
state_avg_arv_AL_LA = pd.read_csv(r"data/pfas_data/state_avg_arv_AL_LA.csv")

# Merge the two datasets on 'state_id' (assuming both have the same 'state_id' column)
merged_data = pd.concat([state_avg_arv, state_avg_arv_AL_LA], ignore_index=True)

# Sort the merged data by 'state_id' in ascending order (this corresponds to the label encoding order)
merged_data_sorted = merged_data.sort_values(by='state_id', ascending=True)

# Save the merged and sorted data to a CSV file
merged_data_sorted.to_csv(r"data/pfas_data/merged_state_avg_arv.csv", index=False)
print("Merged and sorted state average AnalyticalResultValues saved as 'merged_state_avg_arv.csv'.")


Merged and sorted state average AnalyticalResultValues saved as 'merged_state_avg_arv.csv'.
<bound method DataFrame.info of     state_id  AnalyticalResultValue
0          7               0.359763
1          8               0.316810
2          9               0.327840
3         10               0.339042
4         11               0.374170
5         12               0.347457
6         13               0.358817
7         14               0.520991
8         15               0.340402
9         16               0.317085
10        17               0.864648
11        18               0.339540
12         0               2.282289
13         1               0.484683
14         2               0.309330
15         3               0.340777
16         4               0.322682
17         5               0.459714
18         6               0.504917>
